# Hands-on LoRA finetuning tutorial on Orange3 QA & MCQ dataset

### Preparing the environment

In [ ]:
# @title 🛠️ Setup: Clone Repo & Install Dependencies
import os
import sys

# 1. Define your repository
REPO_URL = "https://github.com/MartinSpendl/LoRA-finetuning-tutorial.git"
REPO_NAME = "LoRA-finetuning-tutorial"
branch = "main"

# 2. Clone or Pull (if it already exists)
if os.path.isdir(REPO_NAME):
    print(f"🔄 Updating {REPO_NAME}...")
    !cd {REPO_NAME} && git pull origin {branch}
else:
    print(f"📥 Cloning {REPO_NAME}...")
    !git clone {REPO_URL}

# 3. Add the repo to Python's path so you can import from it
if REPO_NAME not in sys.path:
    sys.path.append(os.path.abspath(REPO_NAME))
print(f"✅ Added '{REPO_NAME}' to Python path.")

# 4. Install dependencies from your repo's requirements.txt
print("📦 Installing dependencies...")
!pip install -q -r {REPO_NAME}/requirements.txt

print("🚀 Ready to go!")

In [ ]:
import os
import torch
import json
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer

import sys
sys.path.append('LoRA-finetuning-tutorial/src')
sys.path.append('../src')
from store_load_results import store_results, load_results
from evaluation_function import evaluate_model
from utils import define_model_name

## PART 1: Basic Finetuning with LoRA using PEFT

### Defining the model config

In [ ]:
MODEL_CONFIG = {
    "base_model": "Qwen/Qwen3-0.6B",
    "finetuning": True,
    "use_dora": True,
    "n_epochs": 1,
    "lora_r": 8,
    "lora_alpha": 16,
    "lr": 5e-4,
    "batch_size": 8,
    "lora_projections": "qvko",
    "lora_dropout": 0.05,
    "new_tokens_path": None,
    "new_tokens_init": "random",
    "new_tokens_train": True,
}

PROJECTIONS = {
    "q": "q_proj",
    "k": "k_proj",
    "v": "v_proj",
    "o": "o_proj",
    "g": "gate_proj",
    "d": "down_proj",
    "u": "up_proj"
}

projections = [PROJECTIONS[p] for p in list(MODEL_CONFIG["lora_projections"])]
MODEL_CONFIG['lora_projections'] = projections
model_name, OUTPUT_DIR = define_model_name(MODEL_CONFIG)

wandb_project = "qwen3-lora-finetuning"
wandb_run_name = model_name
os.environ["WANDB_PROJECT"] = wandb_project

print("Model configuration:")
for key, value in MODEL_CONFIG.items():
    print(f"   {key}: {value}")

## Training dataset

Let's look at the training dataset, and what kind of data we have.

In [ ]:
TRAINDATA_FILE = os.path.join(os.getcwd(), 'LoRA-finetuning-tutorial', 'data', 'train_test_dataset', 'orange_qa_train.jsonl')
TESTDATA_MCQ_FILE = os.path.join(os.getcwd(), 'LoRA-finetuning-tutorial', 'data', 'train_test_dataset', 'orange_qa_MCQ_test.jsonl')
TESTDATA_MCQ_CON_FILE = os.path.join(os.getcwd(), 'LoRA-finetuning-tutorial', 'data', 'train_test_dataset', 'orange_qa_MCQ-con_test.jsonl')

dataset = load_dataset("json", data_files=TRAINDATA_FILE, split="train")

In [ ]:
# check different question types:
# QA: 7
# MCQ: 579
# QA - connection: 0
# MCQ - conection: 1012

ID = 0
sample = dataset[ID]['messages']
for message in sample:
    print("Role:", message['role'])
    print(message['content'])
    print()

### Load the model and tokenizer

Let's get familiar with the model and tokenizer. Specifically look at what layers the model has.

In [ ]:
# 4. Load Model & Tokenizer
print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CONFIG['base_model'])
model = AutoModelForCausalLM.from_pretrained(
    MODEL_CONFIG['base_model'],
    dtype=torch.float16,       # Use float16 to save memory
    device_map="auto",          # Auto-selects GPU or CPU
    do_sample=False,
)

In [ ]:
model

In [ ]:
tokenizer

In [ ]:
# "Orange data mining", "widget", "Hierarchical Clustering", "-->", "-x->"
tokenizer(["Orange Data Mining", "orange data mining"])

### PEFT Module for LoRA/DoRA

In [ ]:
peft_config = LoraConfig(
    bias="none",
    task_type="CAUSAL_LM",
    # from config
    target_modules=MODEL_CONFIG['lora_projections'],
    use_dora=MODEL_CONFIG['use_dora'],
    r=MODEL_CONFIG['lora_r'],
    lora_alpha=MODEL_CONFIG['lora_alpha'],
    lora_dropout=MODEL_CONFIG['lora_dropout'],
    modules_to_save=None,
)
model = get_peft_model(model, peft_config)

In [ ]:
model

In [ ]:
for name, param in model.base_model.model.model.layers[0].self_attn.named_parameters():
    print(name, param.requires_grad)


## Setup training arguments and Train

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=MODEL_CONFIG['n_epochs'],          # How many times to read the docs
    per_device_train_batch_size=MODEL_CONFIG['batch_size'], 
    gradient_accumulation_steps=1,
    learning_rate=MODEL_CONFIG['lr'],
    fp16=True,                   # Use mixed precision
    logging_steps=10,
    optim="adamw_torch",   
    save_strategy="epoch",       # Save a checkpoint every epoch
    report_to=["wandb"],         # Enable wandb logging
    run_name=wandb_run_name,     # Set run name to model name
)

# 8. Initialize Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    processing_class=tokenizer,
)

In [ ]:
trainer.train()

### Evaluate the model on MCQ and MCQ-connection test sets

In [ ]:
# 12. Evaluate and store results
with open(TESTDATA_MCQ_FILE, "r") as f:
    test_mcq_dataset = json.load(f)
accuracy_mcq, se_mcq = evaluate_model(model, tokenizer, test_mcq_dataset, batch_size=MODEL_CONFIG['batch_size'])

with open(TESTDATA_MCQ_CON_FILE, "r") as f:
    test_mcq_con_dataset = json.load(f)
accuracy_mcq_con, se_mcq_con = evaluate_model(model, tokenizer, test_mcq_con_dataset, batch_size=MODEL_CONFIG['batch_size'])
results = {
    "accuracy_mcq": accuracy_mcq,
    "se_mcq": se_mcq,
    "accuracy_mcq_con": accuracy_mcq_con,
    "se_mcq_con": se_mcq_con,
}
print("Evaluation Results:", results)
store_results(results, MODEL_CONFIG)
print("Results stored successfully.")

## PART 2: Token-injection of Orange3 widgets

If you are running out of GPU memory, you can restart the session and run the following cell again.

Just make sure to also run cells from here onwards.

In [ ]:
## Delete if still in memory to free up space
del model
del tokenizer
torch.cuda.empty_cache()

In [ ]:
import os
import torch
import json
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer

import sys
sys.path.append('LoRA-finetuning-tutorial/src')
sys.path.append('../src')
from store_load_results import store_results, load_results
from evaluation_function import evaluate_model
from utils import define_model_name

In [ ]:
MODEL_CONFIG = {
    "base_model": "Qwen/Qwen3-0.6B",
    "finetuning": True,
    "use_dora": True,
    "n_epochs": 1,
    "lora_r": 8,
    "lora_alpha": 16,
    "lr": 5e-4,
    "batch_size": 8,
    "lora_projections": "qvko",
    "lora_dropout": 0.05,
    "new_tokens_path": "LoRA-finetuning-tutorial/data/injected_tokens.json", ## path to new tokens file
    "new_tokens_init": "random",
    "new_tokens_train": True,
}

PROJECTIONS = {
    "q": "q_proj",
    "k": "k_proj",
    "v": "v_proj",
    "o": "o_proj",
    "g": "gate_proj",
    "d": "down_proj",
    "u": "up_proj"
}

projections = [PROJECTIONS[p] for p in list(MODEL_CONFIG["lora_projections"])]
MODEL_CONFIG['lora_projections'] = projections
model_name, OUTPUT_DIR = define_model_name(MODEL_CONFIG)

wandb_project = "qwen3-lora-finetuning"
wandb_run_name = model_name
os.environ["WANDB_PROJECT"] = wandb_project

print("Model configuration:")
for key, value in MODEL_CONFIG.items():
    print(f"   {key}: {value}")

In [ ]:
TRAINDATA_FILE = os.path.join(os.getcwd(), 'LoRA-finetuning-tutorial', 'data', 'train_test_dataset', 'orange_qa_train.jsonl')
TESTDATA_MCQ_FILE = os.path.join(os.getcwd(), 'LoRA-finetuning-tutorial', 'data', 'train_test_dataset', 'orange_qa_MCQ_test.jsonl')
TESTDATA_MCQ_CON_FILE = os.path.join(os.getcwd(), 'LoRA-finetuning-tutorial', 'data', 'train_test_dataset', 'orange_qa_MCQ-con_test.jsonl')

dataset = load_dataset("json", data_files=TRAINDATA_FILE, split="train")

In [ ]:
print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CONFIG['base_model'])
model = AutoModelForCausalLM.from_pretrained(
    MODEL_CONFIG['base_model'],
    dtype=torch.float16,       # Use float16 to save memory
    device_map="auto",          # Auto-selects GPU or CPU
    do_sample=False,
)

### Lets check the tokenizer again, and how it handles some common tokens

In [ ]:
tokenizer

In [ ]:
tokenizer(["Hierarchical Clustering", "-->", "-x->", "Logistic Regression", "Hierarchical Clustering -x-> Logistic Regression"])

### Adding new tokens to tokenizer & Extending model embeddings

In [ ]:
from transformers import AddedToken

# 5. Extend token library (tokenizer + model)
if MODEL_CONFIG['new_tokens_path'] is not None:
    print("Adding new tokens from:", MODEL_CONFIG['new_tokens_path'])
    DEFAULT_TOKEN_NUM = len(tokenizer)

    with open(MODEL_CONFIG['new_tokens_path'], 'r') as f:
        new_tokens_text = json.load(f)
    new_token_tokenized = [tokenizer.encode(token) for token in new_tokens_text]
    new_tokens = [
        AddedToken(token, lstrip=True, rstrip=True)
        for token in new_tokens_text
    ]

    ## extend tokenizer and model
    print("Number of default tokenizer tokens:", DEFAULT_TOKEN_NUM)
    tokenizer.add_tokens(new_tokens)
    model.resize_token_embeddings(len(tokenizer))
    print("Number of new tokenizer tokens:", len(tokenizer))

    new_tokens_ids = [
        tokenizer.encode(token)[0]
        for token in new_tokens_text
    ]
    old_tokens_ids = [id for id in range(DEFAULT_TOKEN_NUM) if id not in new_tokens_ids]

    ## initialize new token embeddings
    model.model.embed_tokens = model.model.embed_tokens.float()

    if MODEL_CONFIG['new_tokens_init'] == "average":
        with torch.no_grad():
            existing_embeddings = model.model.embed_tokens.weight
            for new_id, previous_token_ids in zip(new_tokens_ids, new_token_tokenized):
                model.model.embed_tokens.weight[new_id, :] = existing_embeddings[previous_token_ids].mean(dim=0)
    elif MODEL_CONFIG['new_tokens_init'] == "zero":
        with torch.no_grad():
            model.model.embed_tokens[DEFAULT_TOKEN_NUM:, :].fill_(0.0)
    elif MODEL_CONFIG['new_tokens_init'] == "random":
        pass  # already quazi randomly initialized
    else:
        raise ValueError(f"Unknown new_tokens_init method: {MODEL_CONFIG['new_tokens_init']}")

    ## setup trainable weights for new tokens
    if MODEL_CONFIG['new_tokens_train']:
        def zero_out_old_token_grads(grad):
            new_grad = grad.clone()
            new_grad[old_tokens_ids, :] = 0.0
            return new_grad
        model.model.embed_tokens.weight.requires_grad = True
        model.model.embed_tokens.weight.register_hook(zero_out_old_token_grads)
    model.lm_head.weight.requires_grad = True

### Let's test our tokenizer with new tokens

In [ ]:
tokenizer

In [ ]:
tokenizer(["Hierarchical Clustering", "-->", "-x->", "Logistic Regression", "Hierarchical Clustering -x-> Logistic Regression"])

In [ ]:
if MODEL_CONFIG['new_tokens_path'] is not None:
    modules_to_save = ["lm_head"] + (["embed_tokens"] if MODEL_CONFIG['new_tokens_train'] else [])
else:
    modules_to_save = None

peft_config = LoraConfig(
    bias="none",
    task_type="CAUSAL_LM",
    # from config
    target_modules=MODEL_CONFIG['lora_projections'],
    use_dora=MODEL_CONFIG['use_dora'],
    r=MODEL_CONFIG['lora_r'],
    lora_alpha=MODEL_CONFIG['lora_alpha'],
    lora_dropout=MODEL_CONFIG['lora_dropout'],
    modules_to_save=modules_to_save,
)
model = get_peft_model(model, peft_config)

In [ ]:
print("LM head module requires grad: ", model.base_model.model.lm_head.modules_to_save.default.weight.requires_grad)
print("Embed tokens module requires grad: ", model.base_model.model.model.embed_tokens.modules_to_save.default.weight.requires_grad)

In [ ]:
wandb_project = MODEL_CONFIG.get('wandb_project') or os.environ.get("WANDB_PROJECT", "qwen3-lora-finetuning")
wandb_run_name = MODEL_CONFIG['model_name']

# Set wandb project via environment variable
os.environ["WANDB_PROJECT"] = wandb_project

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=MODEL_CONFIG['n_epochs'],          # How many times to read the docs
    per_device_train_batch_size=MODEL_CONFIG['batch_size'], 
    gradient_accumulation_steps=1,
    learning_rate=MODEL_CONFIG['lr'],
    fp16=True,                   # Use mixed precision
    logging_steps=10,
    optim="adamw_torch",   
    save_strategy="epoch",       # Save a checkpoint every epoch
    report_to=["wandb"],         # Enable wandb logging
    run_name=wandb_run_name,     # Set run name to model name
)

# 8. Initialize Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    processing_class=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
## Training in FP32, now we have to cast back to FP16 to evaluate
model.base_model.model.model.embed_tokens.modules_to_save.default = model.base_model.model.model.embed_tokens.modules_to_save.default.half()
model.base_model.model.lm_head.modules_to_save.default = model.base_model.model.lm_head.modules_to_save.default.half()

In [ ]:
# 12. Evaluate and store results
with open(TESTDATA_MCQ_FILE, "r") as f:
    test_mcq_dataset = json.load(f)
accuracy_mcq, se_mcq = evaluate_model(model, tokenizer, test_mcq_dataset, batch_size=MODEL_CONFIG['batch_size'])

with open(TESTDATA_MCQ_CON_FILE, "r") as f:
    test_mcq_con_dataset = json.load(f)
accuracy_mcq_con, se_mcq_con = evaluate_model(model, tokenizer, test_mcq_con_dataset, batch_size=MODEL_CONFIG['batch_size'])
results = {
    "accuracy_mcq": accuracy_mcq,
    "se_mcq": se_mcq,
    "accuracy_mcq_con": accuracy_mcq_con,
    "se_mcq_con": se_mcq_con,
}
print("\nEvaluation Results:", results)
store_results(results, MODEL_CONFIG)
print("Results stored successfully.")